In [ ]:
@file:DependsOn("ai.timefold.solver:timefold-solver-core:1.14.0")


In [ ]:
import java.time.Duration
import java.time.LocalDateTime
import ai.timefold.solver.core.api.domain.lookup.PlanningId

data class Skill(
    val id: String,
    val name: String
)

data class Location(
    val latitude: Double,
    val longitude: Double
) {
    // Simplified distance calculation (Euclidean)
    fun distanceTo(other: Location): Double {
        val xDiff = latitude - other.latitude
        val yDiff = longitude - other.longitude
        return Math.sqrt(xDiff * xDiff + yDiff * yDiff)
    }

    // Assuming average speed of 50 km/h for travel time calculation
    fun travelTimeTo(other: Location): Duration {
        val distanceKm = distanceTo(other) * 111  // Approximate conversion to kilometers
        val averageSpeedKmh = 50.0
        val timeInHours = distanceKm / averageSpeedKmh
        return Duration.ofMinutes((timeInHours * 60).toLong())
    }
}

data class TimeWindow(
    val start: LocalDateTime,
    val end: LocalDateTime
)

interface Standstill {
    val location: Location
    val departureTime: LocalDateTime?
}


In [ ]:
interface Standstill {
    val location: Location
    val departureTime: LocalDateTime?
}


In [ ]:
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.variable.PlanningVariable
import ai.timefold.solver.core.api.domain.variable.InverseRelationShadowVariable
import ai.timefold.solver.core.api.domain.variable.PlanningVariableGraphType
import ai.timefold.solver.core.api.domain.variable.ShadowVariable

@PlanningEntity
class Meeting(
    @PlanningId
    val id: String,
    override val location: Location,
    val requiredSkills: List<Skill>,
    val timeWindow: TimeWindow,
    val duration: Duration
) : Standstill {

    // Planning variable: previous standstill (Agent or another Meeting)
    @PlanningVariable(valueRangeProviderRefs = ["standstillRange"], graphType = PlanningVariableGraphType.CHAINED)
    var previousStandstill: Standstill? = null

    // Shadow variables
    @ShadowVariable(sourceVariableName = "previousStandstill")
    var arrivalTime: LocalDateTime? = null

    @ShadowVariable(sourceVariableName = "arrivalTime")
    override var departureTime: LocalDateTime? = null

    // For chaining
    @InverseRelationShadowVariable(sourceVariableName = "previousStandstill")
    var nextMeeting: Meeting? = null
}


Line_34.jupyter.kts (22:21 - 63) No value passed for parameter 'variableListenerClass'
Line_34.jupyter.kts (25:21 - 56) No value passed for parameter 'variableListenerClass'

Line_35.jupyter.kts (22:22 - 29) Unresolved reference: Meeting

In [ ]:
import ai.timefold.solver.core.api.score.director.ScoreDirector
import ai.timefold.solver.core.api.domain.variable.VariableListener

class ArrivalTimeUpdatingVariableListener : VariableListener<Schedule, Meeting> {
    override fun beforeEntityAdded(scoreDirector: ScoreDirector<Schedule>?, meeting: Meeting?) {}
    override fun afterEntityAdded(scoreDirector: ScoreDirector<Schedule>?, meeting: Meeting?) {
        updateArrivalTime(scoreDirector, meeting)
    }
    override fun beforeVariableChanged(scoreDirector: ScoreDirector<Schedule>?, meeting: Meeting?) {}
    override fun afterVariableChanged(scoreDirector: ScoreDirector<Schedule>?, meeting: Meeting?) {
        updateArrivalTime(scoreDirector, meeting)
    }
    override fun beforeEntityRemoved(scoreDirector: ScoreDirector<Schedule>?, meeting: Meeting?) {}
    override fun afterEntityRemoved(scoreDirector: ScoreDirector<Schedule>?, meeting: Meeting?) {}

    private fun updateArrivalTime(scoreDirector: ScoreDirector<Schedule>?, meeting: Meeting?) {
        if (meeting == null || scoreDirector == null) return
        val previous = meeting.previousStandstill
        val arrivalTime = if (previous == null) {
            null
        } else {
            val departureTime = previous.departureTime ?: return
            val travelTime = previous.location.travelTimeTo(meeting.location)
            departureTime.plus(travelTime)
        }
        scoreDirector.beforeVariableChanged(meeting, "arrivalTime")
        meeting.arrivalTime = arrivalTime
        scoreDirector.afterVariableChanged(meeting, "arrivalTime")
    }
}

class DepartureTimeUpdatingVariableListener : VariableListener<Schedule, Meeting> {
    override fun beforeEntityAdded(scoreDirector: ScoreDirector<Schedule>?, meeting: Meeting?) {}
    override fun afterEntityAdded(scoreDirector: ScoreDirector<Schedule>?, meeting: Meeting?) {
        updateDepartureTime(scoreDirector, meeting)
    }
    override fun beforeVariableChanged(scoreDirector: ScoreDirector<Schedule>?, meeting: Meeting?) {}
    override fun afterVariableChanged(scoreDirector: ScoreDirector<Schedule>?, meeting: Meeting?) {
        updateDepartureTime(scoreDirector, meeting)
    }
    override fun beforeEntityRemoved(scoreDirector: ScoreDirector<Schedule>?, meeting: Meeting?) {}
    override fun afterEntityRemoved(scoreDirector: ScoreDirector<Schedule>?, meeting: Meeting?) {}

    private fun updateDepartureTime(scoreDirector: ScoreDirector<Schedule>?, meeting: Meeting?) {
        if (meeting == null || scoreDirector == null) return
        val arrivalTime = meeting.arrivalTime ?: return
        val departureTime = arrivalTime.plus(meeting.duration)
        scoreDirector.beforeVariableChanged(meeting, "departureTime")
        meeting.departureTime = departureTime
        scoreDirector.afterVariableChanged(meeting, "departureTime")
    }
}


Line_36.jupyter.kts (4:62 - 70) Unresolved reference: Schedule
Line_36.jupyter.kts (4:72 - 79) Unresolved reference: Meeting
Line_36.jupyter.kts (5:65 - 73) Unresolved reference: Schedule
Line_36.jupyter.kts (5:86 - 93) Unresolved reference: Meeting
Line_36.jupyter.kts (6:64 - 72) Unresolved reference: Schedule
Line_36.jupyter.kts (6:85 - 92) Unresolved reference: Meeting
Line_36.jupyter.kts (9:69 - 77) Unresolved reference: Schedule
Line_36.jupyter.kts (9:90 - 97) Unresolved reference: Meeting
Line_36.jupyter.kts (10:68 - 76) Unresolved reference: Schedule
Line_36.jupyter.kts (10:89 - 96) Unresolved reference: Meeting
Line_36.jupyter.kts (13:67 - 75) Unresolved reference: Schedule
Line_36.jupyter.kts (13:88 - 95) Unresolved reference: Meeting
Line_36.jupyter.kts (14:66 - 74) Unresolved reference: Schedule
Line_36.jupyter.kts (14:87 - 94) Unresolved reference: Meeting
Line_36.jupyter.kts (16:64 - 72) Unresolved reference: Schedule
Line_36.jupyter.kts (16:85 - 92) Unresolved reference: 

In [ ]:
import ai.timefold.solver.core.api.domain.solution.PlanningEntityCollectionProperty
import ai.timefold.solver.core.api.domain.solution.PlanningScore
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.solution.ProblemFactCollectionProperty
import ai.timefold.solver.core.api.domain.valuerange.ValueRangeProvider
import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore

@PlanningSolution
data class Schedule(
    @PlanningEntityCollectionProperty
    val meetings: List<Meeting>,

    @ProblemFactCollectionProperty
    @ValueRangeProvider(id = "agentRange")
    val agents: List<Agent>,

    @ProblemFactCollectionProperty
    @ValueRangeProvider(id = "standstillRange")
    val standstills: List<Standstill>,

    @ProblemFactCollectionProperty
    val skills: List<Skill>
) {
    @PlanningScore
    var score: HardSoftScore? = null
}


Line_37.jupyter.kts (11:24 - 31) Unresolved reference: Meeting
Line_37.jupyter.kts (15:22 - 27) Unresolved reference: Agent

In [ ]:
import kotlin.random.Random
import java.time.LocalDateTime
import java.time.temporal.ChronoUnit

fun generateFakeData(
    numAgents: Int = 5,
    numMeetings: Int = 20,
    numSkills: Int = 3
): Schedule {
    // Define skills
    val skillNames = listOf("Fire Certification", "Bodyguard Certification", "First Aid")
    val skills = skillNames.map { Skill(it.toLowerCase().replace(" ", "_"), it) }

    // Generate agents
    val agents = (1..numAgents).map { index ->
        val agentSkills = skills.shuffled().take(Random.nextInt(1, numSkills))
        Agent(
            id = "agent_$index",
            name = "Agent $index",
            homeLocation = randomLocationInGeneva(),
            skills = agentSkills,
            maxWorkingHoursPerDay = Duration.ofHours(8)
        )
    }

    // Generate meetings
    val meetings = (1..numMeetings).map { index ->
        val requiredSkills = skills.shuffled().take(Random.nextInt(1, numSkills))
        val startTime = LocalDateTime.now().withHour(8).withMinute(0).plusMinutes(Random.nextLong(0, 600))
        val duration = Duration.ofMinutes(Random.nextLong(30, 120))
        val endTime = startTime.plus(duration).plusMinutes(Random.nextLong(0, 120))
        Meeting(
            id = "meeting_$index",
            location = randomLocationInGeneva(),
            requiredSkills = requiredSkills,
            timeWindow = TimeWindow(startTime, endTime),
            duration = duration
        )
    }

    val standstills = agents.map { it as Standstill } + meetings.map { it as Standstill }

    return Schedule(meetings, agents, standstills, skills)
}

// Helper function to generate random locations in Geneva
fun randomLocationInGeneva(): Location {
    // Approximate bounding box for Geneva
    val latMin = 46.188
    val latMax = 46.256
    val lonMin = 6.119
    val lonMax = 6.166
    val latitude = Random.nextDouble(latMin, latMax)
    val longitude = Random.nextDouble(lonMin, lonMax)
    return Location(latitude, longitude)
}


Line_38.jupyter.kts (9:4 - 12) Unresolved reference: Schedule
Line_38.jupyter.kts (12:44 - 55) 'toLowerCase(): String' is deprecated. Use lowercase() instead.
Line_38.jupyter.kts (17:9 - 14) Unresolved reference: Agent
Line_38.jupyter.kts (32:9 - 16) Unresolved reference: Meeting
Line_38.jupyter.kts (41:39 - 41) This cast can never succeed
Line_38.jupyter.kts (41:75 - 77) This cast can never succeed
Line_38.jupyter.kts (43:12 - 20) Unresolved reference: Schedule

In [ ]:
import ai.timefold.solver.core.api.score.stream.Constraint
import ai.timefold.solver.core.api.score.stream.ConstraintFactory
import ai.timefold.solver.core.api.score.stream.ConstraintProvider
import ai.timefold.solver.core.api.score.stream.Joiners

class ScheduleConstraintProvider : ConstraintProvider {

    override fun defineConstraints(constraintFactory: ConstraintFactory): Array<Constraint> {
        return arrayOf(
            skillRequirement(constraintFactory),
            timeWindowViolation(constraintFactory),
            agentWorkingHours(constraintFactory),
            avoidOverlappingMeetings(constraintFactory),
            minimizeTravelDistance(constraintFactory)
        )
    }

    // 1. Agents must have required skills for the meetings they are assigned to.
    fun skillRequirement(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEach(Meeting::class.java)
            .filter { meeting ->
                val agent = getAgent(meeting)
                agent == null || !meeting.requiredSkills.all { skill -> agent.skills.contains(skill) }
            }
            .penalize(HardSoftScore.ONE_HARD)
            .asConstraint("Skill requirement")
    }

    // 2. Meetings must be within their time windows.
    fun timeWindowViolation(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEach(Meeting::class.java)
            .filter { meeting ->
                val arrivalTime = meeting.arrivalTime
                val departureTime = meeting.departureTime
                arrivalTime == null || departureTime == null ||
                        arrivalTime.isBefore(meeting.timeWindow.start) ||
                        departureTime.isAfter(meeting.timeWindow.end)
            }
            .penalize(HardSoftScore.ONE_HARD)
            .asConstraint("Time window violation")
    }

    // 3. Agents must not exceed their working hours per day.
    fun agentWorkingHours(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEach(Agent::class.java)
            .penalize(HardSoftScore.ONE_HARD) { agent ->
                val totalWorkingTime = calculateTotalWorkingTime(agent)
                val overTime = totalWorkingTime.minus(agent.maxWorkingHoursPerDay)
                if (overTime.isNegative) 0 else overTime.toMinutes().toInt()
            }
            .asConstraint("Agent working hours")
    }

    // 4. Avoid overlapping meetings for the same agent.
    fun avoidOverlappingMeetings(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEach(Meeting::class.java)
            .join(Meeting::class.java,
                Joiners.equal { meeting -> getAgent(meeting) },
                Joiners.lessThan { meeting -> meeting.arrivalTime },
                Joiners.overlapping(
                    { meeting -> meeting.arrivalTime },
                    { meeting -> meeting.departureTime },
                    { meeting -> meeting.arrivalTime },
                    { meeting -> meeting.departureTime }
                )
            )
            .penalize(HardSoftScore.ONE_HARD)
            .asConstraint("Avoid overlapping meetings")
    }

    // 5. Minimize total travel distance for agents.
    fun minimizeTravelDistance(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEach(Meeting::class.java)
            .filter { meeting -> meeting.previousStandstill != null }
            .penalizeDouble(HardSoftScore.ONE_SOFT) { meeting ->
                meeting.previousStandstill!!.location.distanceTo(meeting.location)
            }
            .asConstraint("Minimize travel distance")
    }

    // Helper functions
    private fun getAgent(meeting: Meeting): Agent? {
        var standstill = meeting.previousStandstill
        while (standstill != null) {
            when (standstill) {
                is Agent -> return standstill
                is Meeting -> standstill = standstill.previousStandstill
                else -> break
            }
        }
        return null
    }

    private fun calculateTotalWorkingTime(agent: Agent): Duration {
        var totalTime = Duration.ZERO
        var meeting = agent.nextMeeting
        var previousStandstill: Standstill = agent
        while (meeting != null) {
            val travelTime = previousStandstill.location.travelTimeTo(meeting.location)
            val serviceTime = meeting.duration
            totalTime = totalTime.plus(travelTime).plus(serviceTime)
            previousStandstill = meeting
            meeting = meeting.nextMeeting
        }
        // Add travel time back home if needed
        val returnTravelTime = previousStandstill.location.travelTimeTo(agent.homeLocation)
        totalTime = totalTime.plus(returnTravelTime)
        return totalTime
    }
}


Line_39.jupyter.kts (20:34 - 41) Not enough information to infer type variable A
Line_39.jupyter.kts (20:42 - 49) Unresolved reference: Meeting
Line_39.jupyter.kts (21:23 - 30) Cannot infer a type for this parameter. Please specify it explicitly.
Line_39.jupyter.kts (23:64 - 69) Cannot infer a type for this parameter. Please specify it explicitly.
Line_39.jupyter.kts (25:23 - 36) Unresolved reference: HardSoftScore
Line_39.jupyter.kts (31:34 - 41) Not enough information to infer type variable A
Line_39.jupyter.kts (31:42 - 49) Unresolved reference: Meeting
Line_39.jupyter.kts (32:23 - 30) Cannot infer a type for this parameter. Please specify it explicitly.
Line_39.jupyter.kts (39:23 - 36) Unresolved reference: HardSoftScore
Line_39.jupyter.kts (45:34 - 41) Not enough information to infer type variable A
Line_39.jupyter.kts (45:42 - 47) Unresolved reference: Agent
Line_39.jupyter.kts (46:23 - 36) Unresolved reference: HardSoftScore
Line_39.jupyter.kts (46:49 - 54) Cannot infer a type f

In [ ]:
import ai.timefold.solver.core.config.solver.SolverConfig
import ai.timefold.solver.core.api.solver.SolverFactory
import ai.timefold.solver.core.api.solver.Solver

val solverConfig = SolverConfig()
    .withSolutionClass(Schedule::class.java)
    .withEntityClasses(Meeting::class.java)
    .withConstraintProviderClass(ScheduleConstraintProvider::class.java)
    .withTerminationSpentLimit(Duration.ofSeconds(10))

val solverFactory: SolverFactory<Schedule> = SolverFactory.create(solverConfig)

val problem: Schedule = generateFakeData(numAgents = 5, numMeetings = 20, numSkills = 3)

println("Solving the problem ...")
val solver: Solver<Schedule> = solverFactory.buildSolver()
val solution: Schedule = solver.solve(problem)
println("Solving finished with score (${solution.score}).")


Line_40.jupyter.kts (6:24 - 32) Unresolved reference: Schedule
Line_40.jupyter.kts (7:24 - 31) Unresolved reference: Meeting
Line_40.jupyter.kts (8:34 - 60) Unresolved reference: ScheduleConstraintProvider
Line_40.jupyter.kts (11:34 - 42) Unresolved reference: Schedule
Line_40.jupyter.kts (13:14 - 22) Unresolved reference: Schedule
Line_40.jupyter.kts (13:25 - 41) Unresolved reference: generateFakeData
Line_40.jupyter.kts (16:20 - 28) Unresolved reference: Schedule
Line_40.jupyter.kts (17:15 - 23) Unresolved reference: Schedule

In [ ]:
println("Assignments:")
solution.agents.forEach { agent ->
    println("Agent: ${agent.name}")
    var meeting = agent.nextMeeting
    while (meeting != null) {
        println("  Meeting: ${meeting.id}, Arrival: ${meeting.arrivalTime}, Departure: ${meeting.departureTime}, Location: ${meeting.location}")
        meeting = meeting.nextMeeting
    }
}


Line_41.jupyter.kts (2:1 - 9) Unresolved reference: solution
Line_41.jupyter.kts (2:17 - 24) Overload resolution ambiguity: 
public inline fun <T> Iterable<TypeVariable(T)>.forEach(action: (TypeVariable(T)) -> Unit): Unit defined in kotlin.collections
public inline fun <K, V> Map<out TypeVariable(K), TypeVariable(V)>.forEach(action: (Map.Entry<TypeVariable(K), TypeVariable(V)>) -> Unit): Unit defined in kotlin.collections
Line_41.jupyter.kts (2:27 - 32) Cannot infer a type for this parameter. Please specify it explicitly.